<a href="https://colab.research.google.com/github/Nowitkiewicz/PortfolioProjects/blob/main/Bulldozers_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sale Price prediction for bulldozers

In this notebook I am going to predict a sale price of bulldozers in the function of time and using given features.

Problem definition:

How good I'll be able to train a model that predict sale price of bulldozers using data and features downloaded from Kaggle.


Data:

Downloaded from:

Train and valid sets:

https://www.kaggle.com/competitions/bluebook-for-bulldozers/data?select=TrainAndValid.csv

Train.csv is the training set, which contains data through the end of 2011.

Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.


Features:

Downloaded from: https://www.kaggle.com/competitions/bluebook-for-bulldozers/data?select=Data+Dictionary.xlsx
Evaluation:

The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.
*note: my goal is to minimize RMSLE.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import datetime as dt
%matplotlib inline

In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

Saving TrainAndValid.csv to TrainAndValid.csv


In [ ]:
import pandas as pd
import io
path_file_name = input('Enter filename')

train_valid = pd.read_csv(path_file_name)
train_valid

Enter filenameTrainAndValid.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (13,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412693,6333344,10000.0,1919201,21435,149,2.0,2005,NaN,NaN,3/7/2012 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
412694,6333345,10500.0,1882122,21436,149,2.0,2005,NaN,NaN,1/28/2012 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
412695,6333347,12500.0,1944213,21435,149,2.0,2005,NaN,NaN,1/28/2012 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
412696,6333348,10000.0,1794518,21435,149,2.0,2006,NaN,NaN,3/7/2012 0:00,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN


In order to qualify whether or not a sample is train or valid we need to manipulate 'saledate' column, so we change 'saledate

In [ ]:

train_valid['saledate'] = pd.to_datetime(train_valid['saledate'])#, format='%Y/%m/%d')
train_valid['saledate'].dtype

dtype('<M8[ns]')

In [ ]:
# General view on missing labels quantity.
train_valid.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

We are going to copy of the original datasets so all of the changes would not affect oryginal datasets.



In [ ]:
train_valid_copy = train_valid.copy()

Sorting data by 'saledate' to have a better view af sales in a function of time.


In [ ]:
train_valid_copy.sort_values(by=['saledate'],inplace=True)

In [ ]:
train_valid_copy.saledate

205615   1989-01-17
274835   1989-01-31
141296   1989-01-31
212552   1989-01-31
62755    1989-01-31
            ...    
410879   2012-04-28
412476   2012-04-28
411927   2012-04-28
407124   2012-04-28
409203   2012-04-28
Name: saledate, Length: 412698, dtype: datetime64[ns]

Check of actual data structure and types as well as null values.

In [ ]:
train_valid_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 205615 to 409203
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   412698 non-null  int64         
 1   SalePrice                 412698 non-null  float64       
 2   MachineID                 412698 non-null  int64         
 3   ModelID                   412698 non-null  int64         
 4   datasource                412698 non-null  int64         
 5   auctioneerID              392562 non-null  float64       
 6   YearMade                  412698 non-null  int64         
 7   MachineHoursCurrentMeter  147504 non-null  float64       
 8   UsageBand                 73670 non-null   object        
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object        
 11  fiBaseModel               412698 non-null  object        
 1

In [ ]:
train_valid_copy.isna().sum()

SalesID                          0
SalePrice                        0
MachineID                        0
ModelID                          0
datasource                       0
auctioneerID                 20136
YearMade                         0
MachineHoursCurrentMeter    265194
UsageBand                   339028
saledate                         0
fiModelDesc                      0
fiBaseModel                      0
fiSecondaryDesc             140727
fiModelSeries               354031
fiModelDescriptor           337882
ProductSize                 216605
fiProductClassDesc               0
state                            0
ProductGroup                     0
ProductGroupDesc                 0
Drive_System                305611
Enclosure                      334
Forks                       214983
Pad_Type                    331602
Ride_Control                259970
Stick                       331602
Transmission                224691
Turbocharged                331602
Blade_Extension     

As we see we have mostly 'Object' type of data and a few numeric type.We can also see there is a big quantity of missing data.
To start modelling we must change 'Object' type data into numbers ,as well as fill missing values.
One of ways to change 'object' / 'string' type data into numbers is to make them categorical type.



In [ ]:
# Changing 'Object' type columns into category type.
for label,col in train_valid_copy.items():
    if pd.api.types.is_object_dtype(col):
        train_valid_copy[label] = col.astype('category')

In [ ]:
train_valid_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 205615 to 409203
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   412698 non-null  int64         
 1   SalePrice                 412698 non-null  float64       
 2   MachineID                 412698 non-null  int64         
 3   ModelID                   412698 non-null  int64         
 4   datasource                412698 non-null  int64         
 5   auctioneerID              392562 non-null  float64       
 6   YearMade                  412698 non-null  int64         
 7   MachineHoursCurrentMeter  147504 non-null  float64       
 8   UsageBand                 73670 non-null   category      
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  category      
 11  fiBaseModel               412698 non-null  category      
 1

In [ ]:
# Sorting data by "saledate" columns.
train_valid_copy.sort_values(by='saledate',inplace=True)

In [ ]:
# Filling missing data for numeric type with median of numeric columns.
for label,col in train_valid_copy.items():
    if pd.api.types.is_numeric_dtype(col):
        if pd.isnull(col).sum():
            train_valid_copy[label] = col.fillna(col.median())

In [ ]:
# Changing categorical type to numeric values:
for label,col in train_valid_copy.items():
    if label == 'saledate':
        continue
    if not pd.api.types.is_numeric_dtype(col):
        train_valid_copy[label] = pd.Categorical(col).codes + 1

In [ ]:
train_valid_copy.isnull().sum()

SalesID                     0
SalePrice                   0
MachineID                   0
ModelID                     0
datasource                  0
auctioneerID                0
YearMade                    0
MachineHoursCurrentMeter    0
UsageBand                   0
saledate                    0
fiModelDesc                 0
fiBaseModel                 0
fiSecondaryDesc             0
fiModelSeries               0
fiModelDescriptor           0
ProductSize                 0
fiProductClassDesc          0
state                       0
ProductGroup                0
ProductGroupDesc            0
Drive_System                0
Enclosure                   0
Forks                       0
Pad_Type                    0
Ride_Control                0
Stick                       0
Transmission                0
Turbocharged                0
Blade_Extension             0
Blade_Width                 0
Enclosure_Type              0
Engine_Horsepower           0
Hydraulics                  0
Pushblock 

In [ ]:
train_valid_copy.dtypes

SalesID                              int64
SalePrice                          float64
MachineID                            int64
ModelID                              int64
datasource                           int64
auctioneerID                       float64
YearMade                             int64
MachineHoursCurrentMeter           float64
UsageBand                             int8
saledate                    datetime64[ns]
fiModelDesc                          int16
fiBaseModel                          int16
fiSecondaryDesc                      int16
fiModelSeries                        int16
fiModelDescriptor                    int16
ProductSize                           int8
fiProductClassDesc                    int8
state                                 int8
ProductGroup                          int8
ProductGroupDesc                      int8
Drive_System                          int8
Enclosure                             int8
Forks                                 int8
Pad_Type   

Now when we have only numerical values in our dataset,we are able to split set into train and valid sets separately as documentation says.

The split is dependent to the date,telling that untill the end of 2011 data should be in Train.csv
and Valid.csv should be from 1.1.2012 - 30.04.2012.
By using our 'saledate' column we are able to split our train_valid.copy dataframe into train and test sets..

In [ ]:
train_valid_copy.T

,205615,233186,142491,115536,92301,115892,134080,92294,31494,140922,...,407382,405913,405915,411423,412544,409901,405777,411889,411890,409203
SalesID,1646770,1728883,1508502,1452578,1403418,1453054,1491298,1403411,1263797,1504404,...,6267915,6257872,6257884,6309986,6327732,6286556,6256957,6312472,6312473,6283635
SalePrice,9500.0,30000.0,21000.0,33000.0,24000.0,36000.0,46000.0,23000.0,17000.0,26000.0,...,47000.0,13000.0,16500.0,16000.0,11000.0,23000.0,11500.0,11000.0,11000.0,34000.0
MachineID,1126363,1523610,1153157,1544443,1390168,1117184,1322436,1184544,1526902,1310407,...,1844076,1837259,1897564,1899882,1906978,1793587,1860454,1820676,1796787,1869284
ModelID,8434,9105,4138,3854,7110,3854,4107,7110,6953,4139,...,3362,3218,3263,9508,12524,4605,6868,17231,17231,4701
datasource,132,132,132,132,132,132,132,132,132,132,...,149,149,149,149,149,149,149,149,149,149
auctioneerID,18.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
YearMade,1974,1986,1972,1974,1986,1977,1987,1986,1983,1975,...,1977,1995,1996,2005,2006,2006,1000,2006,2006,1000
MachineHoursCurrentMeter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UsageBand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
saledate,1989-01-17 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,...,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00,2012-04-28 00:00:00


In [ ]:
train_valid_copy['YearOfSale'] = train_valid_copy['saledate'].dt.year
train_valid_copy.drop(['saledate'],axis=1,inplace=True)

In [ ]:
train_valid_copy.head().T

,205615,233186,142491,115536,92301
SalesID,1646770.0,1728883.0,1508502.0,1452578.0,1403418.0
SalePrice,9500.0,30000.0,21000.0,33000.0,24000.0
MachineID,1126363.0,1523610.0,1153157.0,1544443.0,1390168.0
ModelID,8434.0,9105.0,4138.0,3854.0,7110.0
datasource,132.0,132.0,132.0,132.0,132.0
auctioneerID,18.0,99.0,99.0,99.0,99.0
YearMade,1974.0,1986.0,1972.0,1974.0,1986.0
MachineHoursCurrentMeter,0.0,0.0,0.0,0.0,0.0
UsageBand,0.0,0.0,0.0,0.0,0.0
fiModelDesc,4593.0,4772.0,2347.0,1767.0,744.0


In [ ]:
train_valid_copy.tail().T

,409901,405777,411889,411890,409203
SalesID,6286556.0,6256957.0,6312472.0,6312473.0,6283635.0
SalePrice,23000.0,11500.0,11000.0,11000.0,34000.0
MachineID,1793587.0,1860454.0,1820676.0,1796787.0,1869284.0
ModelID,4605.0,6868.0,17231.0,17231.0,4701.0
datasource,149.0,149.0,149.0,149.0,149.0
auctioneerID,99.0,99.0,99.0,99.0,99.0
YearMade,2006.0,1000.0,2006.0,2006.0,1000.0
MachineHoursCurrentMeter,0.0,0.0,0.0,0.0,0.0
UsageBand,0.0,0.0,0.0,0.0,0.0
fiModelDesc,500.0,1679.0,581.0,581.0,989.0


Now when all our values are numeric,we are able to fit them to a ML model.

In [ ]:
TrainSet = train_valid_copy[train_valid_copy['YearOfSale'] != 2012]

In [ ]:
ValidationSet = train_valid_copy[train_valid_copy['YearOfSale'] == 2012]

In [ ]:
X_train = TrainSet.drop(['SalePrice'],axis=1)
y_train = TrainSet['SalePrice']
X_valid = ValidationSet.drop(['SalePrice'],axis=1)
y_valid = ValidationSet['SalePrice']

In [ ]:
X_train

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,YearOfSale
205615,1646770,1126363,8434,132,18.0,1974,0.0,0,4593,1744,...,0,0,0,0,1,8,6,0,0,1989
233186,1728883,1523610,9105,132,99.0,1986,0.0,0,4772,1852,...,0,0,0,0,0,0,0,0,0,1989
142491,1508502,1153157,4138,132,99.0,1972,0.0,0,2347,713,...,0,0,0,0,1,1,6,0,0,1989
115536,1452578,1544443,3854,132,99.0,1974,0.0,0,1767,535,...,0,0,0,0,0,0,0,4,2,1989
92301,1403418,1390168,7110,132,99.0,1986,0.0,0,744,245,...,0,0,0,0,0,0,0,0,0,1989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397551,6286845,1818840,8050,149,2.0,2006,0.0,0,493,176,...,0,0,0,0,0,0,0,0,0,2011
397589,6287076,1862704,4640,149,2.0,1000,0.0,0,732,242,...,0,0,0,0,0,0,0,0,0,2011
397660,6287313,1805275,4666,149,2.0,2004,0.0,0,865,267,...,0,0,0,0,1,6,6,0,0,2011
397995,6288403,1908296,26287,149,2.0,1000,0.0,0,1818,557,...,0,0,0,0,0,0,0,0,0,2011


In [ ]:
y_train

205615     9500.0
233186    30000.0
142491    21000.0
115536    33000.0
92301     24000.0
           ...   
397551    34500.0
397589    19000.0
397660    24000.0
397995    18000.0
397022    24000.0
Name: SalePrice, Length: 401125, dtype: float64

In [ ]:
#Import Random Forest Regressor.
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Instantiate and fit a model to a given data.

model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train,y_train)

RandomForestRegressor(n_jobs=-1)

In [ ]:
y_pred =model.predict(X_valid)

In [ ]:
model.score(X_valid,y_valid)

0.8729333459213093

In [ ]:
#Our goal is to minimize the Root Mean Squared log Error...
#Import Mean Squared Log Error.
from sklearn.metrics import mean_squared_log_error

np.sqrt(mean_squared_log_error(y_valid,y_pred))

0.24923145835680163

The required score for Root Mean Squared Log Error:

0.24921126566184412
